In [1]:
import Pkg
Pkg.activate("../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [2]:
using Revise
using Turing
using LinearAlgebra
using Distributions
using Random
using StatsPlots
using PDMats
using StatsBase
using Base.Iterators
using DataFrames, TimeSeries, XLSX
using PrettyTables

In [3]:
include("VARs/utils.jl")
includet("bootstrap_model.jl")

In [4]:
#df = DataFrame(XLSX.readtable("../data/usa_world_data.xlsx", "Arkusz1", infer_eltypes=true))
df = DataFrame(XLSX.readtable("../data/usa_var_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.month, last)

872×11 TimeArray{Float64, 2, Date, Matrix{Float64}} 1951-12-01 to 2024-07-01
┌────────────┬──────────┬───────────┬─────────────┬──────────┬─────────┬────────
│            │ Yeld_10Y │ Price_10Y │ Stock_Price │ Dividend │ E       │ CAPE  ⋯
├────────────┼──────────┼───────────┼─────────────┼──────────┼─────────┼────────
│ 1951-12-01 │  0.02513 │   104.098 │       24.19 │  1.41333 │    2.44 │ 12.52 ⋯
│ 1952-01-01 │  0.02463 │   104.771 │       23.75 │  1.41667 │ 2.42667 │ 12.36 ⋯
│ 1952-02-01 │  0.02617 │   103.585 │       23.81 │     1.42 │ 2.41333 │ 12.36 ⋯
│ 1952-03-01 │  0.02505 │   104.824 │       23.74 │     1.43 │     2.4 │ 12.24 ⋯
│ 1952-04-01 │  0.02422 │   105.806 │       23.73 │     1.44 │    2.38 │ 12.20 ⋯
│ 1952-05-01 │  0.02384 │   106.373 │       24.38 │     1.45 │    2.36 │ 12.44 ⋯
│ 1952-06-01 │  0.02415 │   106.295 │       25.08 │     1.45 │    2.34 │ 12.66 ⋯
│ 1952-07-01 │  0.02466 │   106.034 │       25.18 │     1.45 │ 2.34667 │ 12.67 ⋯
│     ⋮      │    ⋮     │     ⋮ 

In [5]:
#= freq = 12
returns = percentchange(data_source[:ACWI_Net, :USA_Net, :Price_10Y, :World_ex_USA, :EM_Net], :log)
cpi = percentchange(data_source[:CPI], :log)
real_returns = returns .- cpi
real_t_bill = log1p.(data_source[:T_Bill])./freq .- cpi
#= real_10Y = log1p.(data_source[:Yeld_10Y])./freq .- cpi
excess_returns = returns .- log1p.(data_source[:T_Bill]./freq)
dy = log.(data_source[:Dividend_Yeld])
term = log1p.(data_source[:Yeld_10Y] ./ freq)  .- log1p.(data_source[:T_Bill] ./ freq)
term = data_source[:Yeld_10Y]   .- log1p.(data_source[:T_Bill]  =#


start = Date(1988, 01,01)
end_d = Date(2024, 12, 31)


display("Returns")
display(returns_summarystats(to(from(returns, start),end_d), freq))
display("Real_returns")
display(returns_summarystats(to(from(real_returns, start),end_d), freq))
#= display("Excess_returns")
display(returns_summarystats(to(from(excess_returns, start),end_d), freq)) =#

display("Correlations")
display(cor(values(to(from(real_returns, start),end_d))))  =#

In [11]:
freq = 12
returns = percentchange(data_source[:Total_Stock, :Price_10Y], :log)
cpi = percentchange(data_source[:CPI], :log)
real_returns = returns .- cpi
real_t_bill = log1p.(data_source[:T_Bill])./freq .- cpi
excess_returns = returns .- log1p.(data_source[:T_Bill]./freq)
yeld_diff = diff(data_source[:Yeld_10Y])
start = Date(1974, 01,01)
end_d = Date(2024, 12, 31)


display("Returns")
display(returns_summarystats(to(from(returns, start),end_d), freq))
display("Real_returns")
display(returns_summarystats(to(from(real_returns, start),end_d), freq))
display("Excess_returns")
display(returns_summarystats(to(from(excess_returns, start),end_d), freq)) 

display("Correlations")
display(cor(values(to(from(real_returns, start),end_d)))) 

"Returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock,0.1275,1.3903,0.107,0.3744,-2.6994,-1.2795,-0.0945,0.2222,5.5086,0.8393,0.1432
Price_10Y,0.0817,1.3969,0.0623,0.224,-0.9866,0.2076,-0.0999,0.1187,1.5372,0.763,0.0494


nothing

"Real_returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_CPI,0.1284,1.3604,0.0693,0.3413,-2.5776,-1.1894,-0.1274,0.2229,4.7154,0.5394,0.1041
Price_10Y_CPI,0.0847,1.279,0.0246,0.1927,-1.1677,0.1548,-0.1582,0.1652,1.5452,0.2903,0.0184


...

"Excess_returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_T_Bill,0.1279,1.3882,0.0634,0.3345,-2.7107,-1.2674,-0.1381,0.2256,5.2627,0.496,0.1047
Price_10Y_T_Bill,0.0815,1.2656,0.0188,0.1865,-1.1145,0.0905,-0.1602,0.1157,1.4351,0.2303,0.0215


nothing

"Correlations"

2×2 Matrix{Float64}:
 1.0       0.182282
 0.182282  1.0

In [15]:
display(returns_summarystats(to(from(yeld_diff, start),end_d),1))

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Yeld_10Y,0.0034,0.0159,-0.0,0.0018,-0.0188,-0.2394,-0.0018,0.1197,3.4024,-0.0136,0.0


nothing

### Stationary bootstrap
- get singe block
- get multople blocs with static length
- get single blocs with random lengt
- get random block length
- gent random block lengt when, lengt is greater when left length

In [7]:
#assets_names = [:USA_Net_CPI, :Price_10Y_CPI, :World_ex_USA_CPI, :EM_Net_CPI]
#assets_names = [:Total_Stock_CPI, :Price_10Y_CPI]
assets_names = [:Total_Stock_T_Bill, :Price_10Y_T_Bill]
#data = to(from(real_returns[assets_names], start),end_d) 
data = to(from(excess_returns[assets_names], start),end_d) 

607×2 TimeArray{Float64, 2, Date, Matrix{Float64}} 1974-01-01 to 2024-07-01
┌────────────┬────────────────────┬──────────────────┐
│            │ Total_Stock_T_Bill │ Price_10Y_T_Bill │
├────────────┼────────────────────┼──────────────────┤
│ 1974-01-01 │         -0.0313893 │      -0.00776991 │
│ 1974-02-01 │           0.038712 │     -0.000801741 │
│ 1974-03-01 │         -0.0557911 │       -0.0287593 │
│ 1974-04-01 │         -0.0342284 │       -0.0179532 │
│ 1974-05-01 │        -0.00225885 │       0.00908475 │
│ 1974-06-01 │          -0.126481 │      -0.00853625 │
│ 1974-07-01 │         -0.0444461 │       -0.0169384 │
│ 1974-08-01 │          -0.112406 │       -0.0156933 │
│     ⋮      │         ⋮          │        ⋮         │
│ 2024-01-01 │          0.0367513 │       -0.0100599 │
│ 2024-02-01 │          0.0279047 │       -0.0220494 │
│ 2024-03-01 │         -0.0144803 │       0.00317747 │
│ 2024-04-01 │          0.0204779 │       -0.0399888 │
│ 2024-05-01 │          0.0304937 │        0

In [8]:
scenarios = block_bootstrap(values(data), 10_000; block_size=60, random_block=true)
periods = [1,5, 15, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))
n_assets = length(assets_names)
for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, .025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  

,Total_Stock_T_Bill,Price_10Y_T_Bill
1,0.0616,0.0286
5,0.064,0.0258
15,0.0672,0.0258
25,0.0679,0.0257


,Total_Stock_T_Bill,Price_10Y_T_Bill
1,0.1517,0.0841
5,0.0651,0.0358
15,0.0352,0.019
25,0.0262,0.0146


,Total_Stock_T_Bill,Price_10Y_T_Bill
1,-1.0184,0.0231
5,-0.35,-0.57
15,-0.2782,-0.4108
25,-0.1787,-0.3617


,Total_Stock_T_Bill,Price_10Y_T_Bill
1,1.6439,0.4165
5,-0.2647,0.962
15,0.0119,0.2717
25,-0.0189,0.2263


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.4594,-0.2863,-0.2344,-0.0075,0.0824,0.1652,0.2592,0.2897,0.3551
5,-0.0921,-0.0674,-0.0489,0.0202,0.0694,0.1127,0.1611,0.1759,0.1891
15,-0.0215,-0.0059,0.007,0.0444,0.0685,0.092,0.1222,0.131,0.1394
25,0.0035,0.0139,0.0232,0.0508,0.0687,0.086,0.1098,0.117,0.1253


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.1891,-0.1339,-0.1049,-0.029,0.0305,0.0803,0.1645,0.2134,0.237
5,-0.0784,-0.0636,-0.0422,0.0089,0.0292,0.048,0.0789,0.0909,0.1075
15,-0.0237,-0.0154,-0.0078,0.0135,0.028,0.0389,0.054,0.0596,0.0651
25,-0.012,-0.0056,-0.0003,0.0166,0.0267,0.036,0.0476,0.0518,0.0563


Total_Stock_CPI
0.01	0.025	0.05	0.25	0.5	0.75	0.95	0.975	0.99
1	-0.3984	-0.2858	-0.2207	-0.0164	0.084	0.171	0.2888	0.3312	0.377
5	-0.0938	-0.0735	-0.0533	0.0156	0.0759	0.1147	0.1759	0.1916	0.2098
15	-0.0292	-0.015	-0.0038	0.0376	0.0676	0.0946	0.1272	0.1367	0.1485
25	-0.0093	0.0031	0.0124	0.0446	0.0661	0.0867	0.1136	0.1216	0.1308
Price_10Y_CPI
0.01	0.025	0.05	0.25	0.5	0.75	0.95	0.975	0.99
1	-0.1618	-0.1266	-0.1007	-0.0334	0.0171	0.0727	0.1473	0.2179	0.2843
5	-0.0752	-0.0602	-0.0411	-0.0038	0.0194	0.0463	0.0892	0.105	0.1359
15	-0.0327	-0.0248	-0.018	0.0028	0.0205	0.0383	0.0636	0.0727	0.0818
25	-0.022	-0.0164	-0.011	0.0066	0.0203	0.0346	0.0545	0.0607	0.069


In [9]:
quantile(rand(Geometric(1/60), 10_000), [0., .25, .5, .75, 1.])

5-element Vector{Float64}:
   0.0
  16.0
  40.0
  81.0
 637.0

gdy data length 120
n_steps = 60
stsrt_idx = 59

max_end_idx = 1
